# Нахождение меток для data/posts.xls

Для определения меток исходного датасета мы будем использовать
 
предобученную модель  [DeepPavlov/rubert-base-cased](https://huggingface.co/cointegrated/rubert-base-cased-nli-threeway?)

Но сперва обработаем posts.xls

## Предобработка posts.xls



In [1]:
import pandas as pd 

data = pd.read_excel('../data/posts.xlsx')   

In [2]:
data.head()

,id,telegram_id,text,datetime,category_id,channel_id,title,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,1,157314,❗️Восстановление аммиакопровода Тольятти — Оде...,2023-06-07 11:40:06,NaN,-1001394050290,Восстановление аммиакопровода займет до 3 меся...,NaN,NaN,NaN
1,2,18940,Дополнительные 39 миллионов рублей были выделе...,2023-06-07 11:34:56,NaN,-1001098860759,Выделены дополнительные 39 млн рублей на соцуч...,NaN,NaN,NaN
2,3,4868,«Россия уничтожена санкциями»\n ...,2023-06-07 11:03:05,NaN,-1001797434593,«Россия уничтожена санкциями»\n ...,NaN,NaN,NaN
3,4,157315,❗️Правоохранители провели обыски в министерств...,2023-06-07 11:50:06,NaN,-1001394050290,Обыски в минобразования Дагестана по делу о вы...,NaN,NaN,NaN
4,16,157317,"Реконструкция пропускного пункта ""Верхний Ларс...",2023-06-07 12:10:25,NaN,-1001394050290,"Реконструкция пропускного пункта ""Верхний Ларс...",NaN,NaN,NaN


In [4]:
data['text'].describe()

count                                                169871
unique                                               157940
top       This channel is unavailable due to copyright i...
freq                                                    166
Name: text, dtype: object

In [13]:
# Удалим дубликаты
data = data.drop_duplicates(subset=['text'], keep=False, ignore_index=True)
data.shape

(151531, 10)

In [11]:
data['text'].describe()

count                                                151531
unique                                               151531
top       ❗️Восстановление аммиакопровода Тольятти — Оде...
freq                                                      1
Name: text, dtype: object

In [12]:
data['text'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 151531 entries, 0 to 151530
Series name: text
Non-Null Count   Dtype 
--------------   ----- 
151531 non-null  object
dtypes: object(1)
memory usage: 1.2+ MB


In [15]:
data.to_excel('../data/handled_posts.xlsx', index=False)

In [17]:
sum(pd.isnull(data['text']))

0

## Вычисление меток

In [28]:
! pip install transformers sentencepiece --quiet
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-base-cased-nli-threeway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()


[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
def predict_zero_shot(text, label_texts, model, tokenizer, label='entailment', normalize=True):
    label_texts
    tokens = tokenizer([text] * len(label_texts), label_texts, truncation=True, return_tensors='pt', padding=True)
    with torch.inference_mode():
        result = torch.softmax(model(**tokens.to(model.device)).logits, -1)
    proba = result[:, model.config.label2id[label]].cpu().numpy()
    if normalize:
        proba /= sum(proba)
    return proba

In [30]:
classes = ['Финансы', 'Технологии', 'Политика', 
           'Шоубиз', 'Fashion', 'Крипта', 'Путешествия/релокация', 
           'Образовательный контент', 'Развлечения', 'Общее']

In [ ]:
from numpy import argmax
from tqdm import tqdm

import csv


unhandled_posts = []

with open('../data/labels.csv', 'w', encoding='cp1251') as f:
    writer = csv.writer(f)

    for indx, paper in enumerate(tqdm(data['text'])):
        try:
            proba = predict_zero_shot(paper, classes, model, tokenizer)
            writer.writerow(classes[argmax(proba)])
        except:
            unhandled_posts.append(indx)
        
